# Creation of dimension

The purpose here is to extract the different dimension of the final_result file

In [246]:
import pandas as pd
import numpy as np
import random as rd
pd.options.display.max_columns = 400
print('The pandas version is {}.'.format(pd.__version__))
import hashlib
from rdgeneration import Rdgen
pd.options.mode.chained_assignment = None 
import os

The pandas version is 0.23.4.


## Import file

In [247]:
df = pd.read_csv('../results/final_res.csv')

## Dim Etudiant

In [277]:
def get_ind(var,strr,dic,unique,printed):
    if((var[printed] in unique) and (var[printed] not in dic) ):
        dic[var[printed]] = var[strr]
        
def get_ind2(df):
    df.drop('index', axis = 1,inplace=True)
    df.drop_duplicates(inplace=True)
    df.reset_index(inplace=True)
    return df

In [249]:
df["time_year"] = df["N° Etudiant"].apply(lambda var: var.split("_")[0])
df["codeCS"] = df["code_UE"].apply(lambda var : var.split('-')[0])
df["idcss"] = df.loc[:,["responsable_UE","codeCS"]].apply(lambda var : var["codeCS"] + "-" +var["responsable_UE"], axis=1)
df["semestre"] = df["code_UE"].apply(lambda var : var.split("-")[2])
df["unicite"] = df.loc[:,["time_year","semestre"]].apply(lambda var : var["time_year"]+"-"+var["semestre"],axis = 1)

In [250]:
df.drop("Unnamed: 0", inplace=True, axis=1)
df.reset_index(inplace= True)

### Definition des jetons

On regarde libelleUE, codeCS et semestre pour avoir l'ensemble des UE et affecter les Jac aux differentes CS

Au vu des résultats, les UVs ne sont pas pareils en fonction des années. Donc une CS pour 2015 peut être évalué par 4 UV et en 2016 par 2 seulement et en 2017 par 3. <br>
On va établir nos jetons a partir du minimum d'UV validant les CS. C'est à dire que pour l'exemple précédent, l'UV va être validé pour la validation de 2 UV et ce pour chaque année. <br>
Toutes les UV donnent 4 jetons. (choisis aléatoirement)

In [7]:
forj = df[["nom_UV","codeCS","semestre"]].drop_duplicates()

In [8]:
forj = df[["nom_UV","codeCS","semestre","time_year"]]

In [9]:
forj_2015 = forj[forj["time_year"] == "2015"].drop_duplicates()
forj_2016 = forj[forj["time_year"] == "2016"].drop_duplicates()
forj_2017 = forj[forj["time_year"] == "2017"].drop_duplicates()

In [10]:
forj_2015_s1 = forj_2015[forj_2015["semestre"] == "S1"].sort_values("codeCS")
forj_2015_s2 = forj_2015[forj_2015["semestre"] == "S2"].sort_values("codeCS")
forj_2016_s1 = forj_2016[forj_2016["semestre"] == "S1"].sort_values("codeCS")
forj_2016_s2 = forj_2016[forj_2016["semestre"] == "S2"].sort_values("codeCS")
forj_2017_s1 = forj_2017[forj_2017["semestre"] == "S1"].sort_values("codeCS")
forj_2017_s2 = forj_2017[forj_2017["semestre"] == "S2"].sort_values("codeCS")

In [11]:
def sdf(annee,semestre,cs):
    dff = globals()["forj_"+str(annee)+"_"+str(semestre)]
    return dff[dff["codeCS"] == cs]

In [12]:
sdf(2017,'s1','STC')

,nom_UV,codeCS,semestre,time_year
9379,Calcul scientifique,STC,S1,2017


In [13]:
df["codeCS"].unique()

array(['IGA', 'IGB', 'interpA', 'interpB', 'intraA', 'intraB', 'intraC',
       'STA', 'STB', 'STC'], dtype=object)

Après observation pour jac, on decide :
- interpA interpB intraA intraB intraC : 4 sur l'année

Car ce sont des matirèes qui ne sont pas pris par tout les étudiants.<br>

Pour le reste: 
2015 S1 :
- STA 12
- STB 12
- STC 4

2015 S2:
- STA 16
- STB 4
- STC 12
- IGA 4
- IGB 4

2016 S1:
- STA 12
- STB 12
- STC 4

2016 S2:
- STA 12
- STB 4
- STC 12

2017 S1:
- STA 8
- STB 8
- STC 4

2017 S2:
- STA 8
- STB 8
- STC 4

In [251]:
jacq_dic = {"2015" : { "S1" : 
                      {"STA" : 12,
                       "STB" : 12,
                       "STC" : 4}, 
                      "S2" : 
                      {"STA" : 16,
                       "STB" : 4,
                       "STC" : 12, 
                       "IGA" : 4,
                       "IGB" : 4}
                     },
            "2016" : { "S1" : 
                      {"STA" : 12,
                       "STB" : 12,
                       "STC" : 4}, 
                      "S2" : 
                      {"STA" : 12,
                       "STB" : 4,
                       "STC" : 12}
                     },
            "2017" : { "S1" : 
                      {"STA" : 8,
                       "STB" : 8,
                       "STC" : 4}, 
                      "S2" : 
                      {"STA" : 8,
                       "STB" : 8,
                       "STC" : 4}
                     }
            
           }

## Dim Temps

In [342]:
df_tps = df.loc[:,["index","time_year","semestre"]]
df_tps["unicite"] = df.loc[:,["time_year","semestre"]].apply(lambda var : var["time_year"]+"-"+var["semestre"],axis = 1)

df_tps = get_ind2(df_tps)
df_tps.drop("index",axis=1,inplace=True)
df_tps.reset_index(inplace=True)
df_tps.drop("index",axis=1,inplace=True)
df_tps.reset_index(inplace=True)

col = ['time_id', 'annee', 'semestre', 'unicite']
df_tps.columns = col

df["time_id"] = df.apply(lambda var : df_tps[df_tps["unicite"] == var["unicite"]].index[0],axis = 1)
df_tps.drop("unicite",inplace= True,axis=1)

## Dim Etudiant

In [351]:
df_student = df.loc[:,["index","N° Etudiant","Nom","Prénom","time_year"]]
df_student = get_ind2(df_student)

df_student.drop("index",axis=1,inplace=True)
df_student.reset_index(inplace=True)
df_student.drop("index",axis=1,inplace=True)
df_student.reset_index(inplace=True)

col = list(df_student.columns)
col[0] = "std ID"
df_student.columns = col

df["std ID"] = df.apply(lambda var : df_student[df_student["N° Etudiant"] == var["N° Etudiant"]].index[0],axis = 1)

## Dim UV

In [325]:
df_UV = df.loc[:,["index","nom_UV","code_UV","time_year","semestre"]]
df_UV = get_ind2(df_UV)
df_UV.drop("index",axis=1,inplace=True)
df_UV.reset_index(inplace=True)
df_UV.drop("index",axis=1,inplace=True)
df_UV.reset_index(inplace=True)

col = list(df_UV.columns)
col[0] = "uv ID"
df_UV.columns = col

df["UV ID"] = df.apply(lambda var : df_UV[(df_UV["nom_UV"] == var["nom_UV"]) & (df_UV["code_UV"] == var["code_UV"]) & (df_UV["time_year"] == var["time_year"]) & (df_UV["semestre"] == var["semestre"])]["uv ID"].index[0],axis = 1)

## Dim CS

In [357]:
df_cs = df.loc[:,["index","idcss","responsable_UE","libelle_UE","codeCS","semestre","time_year"]]
df_cs = get_ind2(df_cs)
df_cs.drop("index",axis=1,inplace=True)
df_cs.reset_index(inplace=True)
df_cs.drop("index",axis=1,inplace=True)
df_cs.reset_index(inplace=True)

col = ['cs ID', 'idcss', 'respoCS', 'nom_CS', 'codeCS','semestre','annee']
df_cs.columns = col

df["cs ID"] = df.apply(lambda var : df_cs[(df_cs["idcss"] == var["idcss"]) & (df_cs["semestre"] == var["semestre"]) & (df_cs["annee"] == var["time_year"])].index[0],axis = 1)

df_cs.drop("idcss",inplace= True,axis=1)
mandatory = ['STA','STB','STC','IGA','IGB']
df_cs["jacq"] = df_cs.apply(lambda line : jacq_dic[line["annee"]][line["semestre"]][line["codeCS"]] if line["codeCS"] in mandatory else 4 ,axis = 1)

In [294]:
df_cs.columns

Index(['cs ID', 'respoCS', 'nom_CS', 'codeCS', 'semestre', 'annee', 'jacq'], dtype='object')

## Dim CG

In [366]:
rel_CG_nom = {"IG": "Compétences en ingénierie",
             "interp": "Compétences interpersonnelles",
             "intra" : "Compétences intra-personnelles",
             "ST" : "Compétences scientifiques et techniques"}
df_cg = df.loc[:,["index","codeCS"]]
df_cg["codeCS"] = df_cg["codeCS"].apply(lambda var: var[0:-1])
df_cg["cg_nom"] = df_cg["codeCS"].apply(lambda var: rel_CG_nom[var])

df_cg = get_ind2(df_cg)

df_cg.drop("index",axis=1,inplace=True)
df_cg.reset_index(inplace=True)
df_cg.drop("index",axis=1,inplace=True)
df_cg.reset_index(inplace=True)

col = ["cg id","code cg","cg_nom"]
df_cg.columns = col


df["cg ID"] = df.apply(lambda var : df_cg[df_cg["code cg"] == var["codeCS"][:-1]].index[0],axis = 1)


In [297]:
df_cg.columns

Index(['cg id', 'code cg', 'cg_nom'], dtype='object')

# Degenerated Dim UvGrade

In [33]:
temp = {'id' : [0,1,2,3,4,5], 'symbol' : ['+','=','-','','ABS','DIS']}
uv_grade = pd.DataFrame(temp)

In [34]:
dic_temp = {'+':0,'=':1,'-':2, np.nan:3,'ABS':4,'DIS':5}

In [35]:
df['uv_grade_ID'] = df.apply(lambda var : dic_temp[var['note']], axis =1)

# Dim Grade ECTS

In [37]:
df["Grade"].unique()

array(['atteint', 'en dessous', 'au dessus', nan], dtype=object)

In [38]:
temp = {'id' : [0,1,2,3,4,5,6,7], 
        'grade_ECTS' : ['A','B','C','D','E','F','FX',''], 
        'grade_atteint' : ['au dessus','atteint','atteint','atteint','atteint','en dessous','en dessous','']}
cpt_grade = pd.DataFrame(temp)

In [39]:
dic_temp = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'FX':6,np.nan:7}

In [40]:
df['cpt_grade_ID'] = df.apply(lambda var: dic_temp[var['Grade ECTS']],axis =1)

# Dim Degenerated Lieu

In [60]:
temp = {'id' : [0], 'lieu': ['Nantes']}
lieu_d = pd.DataFrame(temp)

In [61]:
dic_temp = {'Nantes': 0}
df['lieu_ID'] = df.apply(lambda var : dic_temp[var['lieu']],axis =1)

In [181]:
lieu_d.columns

Index(['id', 'lieu'], dtype='object')

## Fact Table TDFNotes

In [185]:
def jnote(line):
    if line["note"] == "+":
        return 5
    elif line["note"] == "=":
        return 4
    else: 
        return 0

In [186]:
ft_notes = df.loc[:,["cs ID","UV ID","time_id","std ID","lieu_ID","note"]]

In [187]:
ft_notes["jetons_acquis"] = ft_notes.apply(lambda line: jnote(line), axis = 1)

In [188]:
ft_notes.drop("note",inplace = True, axis = 1)

## Fact Table TDF_csuv

In [131]:
ft_csuv = df.loc[:,["cs ID","UV ID"]]

In [164]:
def niv_cs(line):
    tmp = df_cs[df_cs["cs ID"] == line["cs ID"]]

    if tmp["semestre"][tmp.index[0]] == "S1":
        return 1
    elif tmp["semestre"][tmp.index[0]] == "S2" :
        return 2

In [165]:
ft_csuv["niveau_CS"] = ft_csuv.apply(lambda line : niv_cs(line),axis = 1)

In [166]:
ft_csuv["nb_jetons"] = 4

## Fact Table TDF_cscg

In [168]:
df.head(1)

,index,N° Etudiant,Nom,Prénom,Moyenne,Grade,Grade ECTS,code_UE,libelle_UE,responsable_UE,annee,nom_UV,code_UV,note,lieu,time_year,codeCS,idcss,semestre,unicite,time_id,std ID,lieu_ID,cs ID,cg ID,UV ID
0,0,2015_77ce,Knapp,Casey,10.0,atteint,B,IGA-A1-S2,IGA : Pratique de l'ingénierie,GOSSIAUX Pol Bernard,1ère année d'ingénieur 2014-2015,PRIME,1A-S2-PRI,=,Nantes,2015,IGA,IGA-GOSSIAUX Pol Bernard,S2,2015-S2,0,0,0,0,0,0


In [170]:
ft_cscg = df.loc[:,["cs ID","cg ID"]]

In [173]:
ft_cscg = ft_cscg.drop_duplicates()

# Creation data frame

In [ ]:
import itertools


In [ ]:
l = 100000
idd = list(itertools.chain.from_iterable([[k,k,k] for k in range(l)]))

In [ ]:
tomax = [1,1,3,2,2,4,1,2,1,4,4,1,5,9,5]*int(l/5)

In [ ]:
junk = ['a']*(3*l)

In [ ]:
mydic = {"id": idd, "tomax": tomax, "junk": junk}
mydf = pd.DataFrame(mydic)

In [ ]:
unique = mydf['id'].unique()

In [ ]:
my_ind = mydf.groupby(['id'])['tomax'].idxmax()
mydf.iloc[my_ind]